# DC Residential Properties Data Analysis

**Author:** Nick Solovyev <br>
**Email:** sonic1@umbc.edu <br>

**Dataset:** [DC Residential Properties | Kaggle](https://www.kaggle.com/christophercorrea/dc-residential-properties) <br>
**Last Edited:** 1/2/19 

## Problem Description

- Regression problem


- I will attempt to predict D.C. housing prices from a dataset of residential data in D.C.

## Dataset

- Link to dataset source: [DC Residential Properties | Kaggle](https://www.kaggle.com/christophercorrea/dc-residential-properties)


- <b>Label/target</b> description - I am trying to predict the price of a house in D.C. This is the 'PRICE' attribute in the dataframe


- Feature description: 
    - Number of rooms (also number of bathrooms, bedrooms, fireplaces, other quantifiable attributes)
    - Geographic location (Latitude and Longitude)
    - Sqr footage of the house (also sqr footage of land)
    - When the house was built, remodeled, additions were built on
    - Categorical features qualifying condition of house, walls, roof, heating, style, etc.
    - Categorical feature qualifying part of the city house is located in
    - Other less useful attributes like address, zipcode, building number


## Import Data

In [1]:
# Import necessary modules for loading the dataset
import os
import pandas as pd

#Load the data
DATA_FILE = 'DC_Properties.csv'
if not os.path.exists(DATA_FILE):
    raise Exception('Data file not found. Make sure that the file is located in the same directory as the notebook')

df = pd.read_csv(DATA_FILE, sep=',', header=0, index_col=0)

# Basic overview of data shape, size, and type
df.info()

# Print data shape via built-in methods of sklearn, pandas or tensorflow/keras (or other modules)
print('\nDataframe shape: ', df.shape)

C:\Users\solon\Anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 158957 entries, 0 to 158956
Data columns (total 48 columns):
BATHRM                158957 non-null int64
HF_BATHRM             158957 non-null int64
HEAT                  158957 non-null object
AC                    158957 non-null object
NUM_UNITS             106696 non-null float64
ROOMS                 158957 non-null int64
BEDRM                 158957 non-null int64
AYB                   158686 non-null float64
YR_RMDL               80928 non-null float64
EYB                   158957 non-null int64
STORIES               106652 non-null float64
SALEDATE              132187 non-null object
PRICE                 98216 non-null float64
QUALIFIED             158957 non-null object
SALE_NUM              158957 non-null int64
GBA                   106696 non-null float64
BLDG_NUM              158957 non-null int64
STYLE                 106696 non-null object
STRUCT                106696 non-null object
GRADE                 106696 non-null

In [2]:
# Sample of the data
df.head()

,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,EYB,...,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT
0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,1972,...,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,1972,...,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,1984,...,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,1984,...,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,1985,...,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW


## Data Visualization

In [3]:
# 

## Data Preprocessing

In [4]:
# Import necessary preprocessing modules
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

In [5]:
# Don't want to impute values for label column as that will hinder model performance
df.dropna(subset=['PRICE'], inplace=True)

# I also want to get rid of unqualified sales. 
# Unqualified sales are not reflective of market value (usually done between family)
df = df[df['QUALIFIED'] != 'U']

In [6]:
# Some feature engineering
df['bedrooms_per_room'] = df['BEDRM'] / df['ROOMS']
df['stories_per_gba'] = df['STORIES'] / df['GBA']
df['free_land'] = df['LANDAREA'] - df['GBA']
df['new_ayb'] = df['AYB'].apply(lambda x: 2019 - x)
df['new_eyb'] = df['EYB'].apply(lambda x: 2019 - x)
                             
df.info
corr_matrix = df.corr()
corr_matrix['PRICE'].sort_values(ascending=False)

PRICE                1.000000
LIVING_GBA           0.700501
GBA                  0.657826
BATHRM               0.578056
BEDRM                0.429516
ROOMS                0.420926
LANDAREA             0.404040
HF_BATHRM            0.333991
CMPLX_NUM            0.239359
EYB                  0.238357
free_land            0.237809
bedrooms_per_room    0.200992
SALE_NUM             0.179705
YR_RMDL              0.161042
Y                    0.155345
LATITUDE             0.155314
FIREPLACES           0.082614
new_ayb              0.067379
KITCHENS             0.058373
BLDG_NUM             0.048475
STORIES              0.034364
NUM_UNITS            0.005934
USECODE             -0.058690
stories_per_gba     -0.060337
AYB                 -0.067379
ZIPCODE             -0.080647
new_eyb             -0.238357
CENSUS_TRACT        -0.305784
X                   -0.346459
LONGITUDE           -0.346499
Name: PRICE, dtype: float64

In [7]:
# Drop poor features
df.drop(['QUALIFIED', 
         'SALE_NUM', 
         'SALEDATE',
         'BLDG_NUM', 
         'GIS_LAST_MOD_DTTM', 
         'SOURCE', 
         'FULLADDRESS', 
         'CMPLX_NUM',
         'CITY',
         'STATE',
         'ZIPCODE',
         'LATITUDE',
         'LONGITUDE',
         'ASSESSMENT_SUBNBHD',
         'NUM_UNITS',
         'LANDAREA',
         'SQUARE',
         'CENSUS_BLOCK',
         'NATIONALGRID',
         'YR_RMDL',
         'EYB',
         'AYB',
         'USECODE'], 
        axis=1,
        inplace=True)

In [8]:
# Get names of categorical and numerical features
cat_df_list = list(df.select_dtypes(include=['object']))
num_df_list = list(df.select_dtypes(include=['float64', 'int64']))
num_df_list.remove('PRICE') # Don't want labels in my list of features

# I added an imputer to my categorical data pipeline and commented out the loop that removes Null values
# May uncomment if imputing adds too much bias

# # Get rid of houses that have Null categorical features
# for feature in cat_df_list:
#     df.dropna(subset=[feature])
    
# df.info()

In [9]:
# pipeline for numerical features
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
    
])

# pipeline for categorical features
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder()),
])

X_pipeline = ColumnTransformer([
    ('cat', cat_pipeline, cat_df_list),
    ('num', num_pipeline, num_df_list),
])

In [10]:
X = df.drop(columns=['PRICE'])
y = df['PRICE']

# Split to train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prep data with pipeline
X_prepared = X_pipeline.fit_transform(X)  # Whole set ran through pipeline for cross-val
X_train_prepared = X_pipeline.transform(X_train)
X_test_prepared = X_pipeline.transform(X_test)

### Trying Linear Regression

In [11]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics

lin_reg = LinearRegression()
lin_reg.fit(X_train_prepared, y_train)

preds = lin_reg.predict(X_train_prepared)
lin_reg_RMSE = np.sqrt(metrics.mean_squared_error(y_train, preds))

print('Training RMSE Score - Linear Regression: ', lin_reg_RMSE)

lin_reg_cv_score = cross_val_score(lin_reg, X_prepared, y.values, cv=10, scoring='neg_mean_squared_error')
print('Mean Cross Val RMSE Score - Linear Regression: ', np.sqrt(lin_reg_cv_score.mean() * -1))

Training RMSE Score - Linear Regression:  270812.3416453154
Mean Cross Val RMSE Score - Linear Regression:  294488.6919503783


A Linear model is too simple for this dataset. Let's try adding polynomial features to the dataset. <br>
To do this, I will create a new datapipeline

### Trying Polynomial Regression

In [12]:
from sklearn.preprocessing import PolynomialFeatures


# pipeline for numerical features turned to polynomials
poly_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('poly_features', PolynomialFeatures(degree=5, include_bias=False)),
    ('std_scaler', StandardScaler()),
    
])

# pipeline for categorical features
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder()),
])

# transformer for polynomial features
X_poly_pipeline = ColumnTransformer([
    ('cat', cat_pipeline, cat_df_list),
    ('num', poly_pipeline, num_df_list),
])

In [ ]:
# Prep data with pipeline

# Pipeline takes an excessively long time to transform data
# Has not been able to complete on my machine
X_poly = X_poly_pipeline.fit_transform(X)  # Whole set ran through pipeline for cross-val
X_train_poly = X_poly_pipeline.transform(X_train)
X_test_poly = X_poly_pipeline.transform(X_test)

poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train)

preds = lin_reg.predict(X_train_poly)
poly_reg_RMSE = np.sqrt(metrics.mean_squared_error(y_train, preds))

print('Training RMSE Score - Polynomial Regression: ', poly_reg_RMSE)

poly_reg_cv_score = cross_val_score(poly_reg, X_poly, y.values, cv=10, scoring='neg_mean_squared_error')
print('Mean Cross Val RMSE Score - Polynomial Regression: ', np.sqrt(poly_reg_cv_score.mean() * -1))

In [ ]:
# I was trying PCA to see if I can reduce dimensionality of problem
# Ran into type problem: PCA doesn't work with sparse matrices

# from sklearn.decomposition import PCA
# pca = PCA(n_components = 0.95)
# X_reduced = pca.fit_transform(X_train_prepared)

# print(X_reduced.shape)